# Debugging

Debugging mode can be turned on globally by setting the `FEEDBAX_DEBUG` environment variable to `"True"`. 

This can be done from within Python, as shown below, or by setting `FEEDBAX_DEBUG=True` in your shell before executing Python from the command line.

In [1]:
import os

os.environ["FEEDBAX_DEBUG"] = str(True)

## Logging details of model execution

The main debugging feature implemented in Feedbax so far is automatic logging of the state operations performed by any subclass of `AbstractStagedModel`. 

For example, now that we've enabled debugging, when a model is executed during training or validation...

In [2]:
import jax

from feedbax.xabdeef import point_mass_nn_simple_reaches


key_init, key_eval = jax.random.split(jax.random.PRNGKey(0))
context = point_mass_nn_simple_reaches(key=key_init)
task, model = context.task, context.model

# We won't bother to train the model for this example

states = task.eval(model, key=key_eval)

...a detailed summary of the model stages that were executed will be saved to the log. 

In [4]:
!cat feedbax.log

2024-02-09 17:44:22,621 [INFO] feedbax,34: Logger configured.
2024-02-09 17:44:26,207 [DEBUG] feedbax.staged,135: 
  Module: SimpleFeedback
  Stage: update_feedback
  Stage module:
      MultiModel(
        models=[
          Channel(
            delay=1,
            noise_std=0.0,
            init_value=nan,
            input_proto=(f32[2], f32[2]),
            intervenors={'update_queue': [], 'add_noise': []}
          )
        ]
      )
  Input:
      [(f32[2], f32[2])]
  Substate:
      [
        ChannelState(
          output=(f32[2], f32[2]),
          queue=((f32[2], f32[2]),),
          noise=(f32[2], f32[2])
        )
      ]

2024-02-09 17:44:26,210 [DEBUG] feedbax.staged,135: 
  Module: Channel
  Stage: update_queue
  Stage module:
      BoundMethod(
        __func__=<function _update_queue>,
        __self__=Channel(
          delay=1,
          noise_std=0.0,
          init_value=nan,
          input_proto=(f32[2], f32[2]),
          intervenors={'update_queue': [], 'add_

Each entry in the log begins with the date and time the log was made. 

!!! WARNING     
    Keep in mind that for long training runs, enabling `FEEDBAX_DEBUG` may result in very large log files, with all of the model stages logged repeatedly on every training run. If you are trying to debug a structural problem in the model step, it's preferable to do a single evaluation of the model as above, so that each stage will only be executed once.

Entries whose first line contains `[DEBUG] feedbax.staged` are logs of model stages that were just executed. Thus, the time between the previous log entry and the current one gives the approximate duration of the current stage.

Following that first line are indented lines that provide details:

- **Model type** is the type of `AbstractStagedModel` that the stage just executed belongs to. A staged model often contains multiple stages that are executed in series, so there may be multiple consecutive log entries with the same model type.
- **Stage** is the stage's label, which serves to distinguish it among all the stages that belong to its model type.
- **Callable** is what is actually called, in order to execute the stage. This may be 
    - an Equinox model---such as another type of `AbstractStagedModel`---in which case the code that was executed can be found in that module's `__call__` method. For example, in one of the stages logged above, the Equinox module `GRUCell` was [called](https://github.com/patrick-kidger/equinox/blob/34252989c251c315c6eee5895dd64e9645b350ce/equinox/nn/_rnn.py#L86).
    - a method of an class or module. Typically these are indicated as `BoundMethod`, where `__func__` provides the name of the method, and `__self__` provides the class that the method belongs to.
    - some other function or callable object.
- **Input** gives the structure of the input that was passed to the callable.
- **Substate** gives the part of the state, associated with the model type, that is passed to the callable along with the input. It has the same structure as the return value of the callable, which is used to update the same part of the model state. 
    - For example, the state associated with the staged model `SimpleFeedback` is `SimpleFeedbackState`. This state contains a field `network` of type `NetworkState`. The stage `"nn_step"` of `SimpleFeedback` passes this `NetworkState` to `SimpleNetwork`, which returns an updated `NetworkState`.
    


## Debugging in JAX

JAX provides several debugging [features](https://jax.readthedocs.io/en/latest/debugging/index.html) which are useful if you'll be developing models with. 